# This is a short notebook to show sample code for my master's application to MILA. For this sample, I decided to show case the training of a DenseNet model. 

## Thank you for taking the time to read this. Feel free to reach out if you have any questions or comments regarding this. 

#### Vincent Dandenault - vincent.dandenault@polymtl.ca 

## Import

In [1]:
%capture 
!pip install IPython 

UsageError: Line magic function `%capture` not found (But cell magic `%%capture` exists, did you mean that instead?).


In [2]:
%autosave 30

Autosaving every 30 seconds


In [3]:
!pip3 install keras_tqdm
!pip3 install utils2
!pip3 install keras
!pip3 install tensorflow
!pip3 install setuptools
!pip3 install keras 
!pip3 install -U scikit-learn
!pip3 install numpy
!pip3 install scipy

  Created wheel for utils2: filename=utils2-0.1.2-py2.py3-none-any.whl size=5241 sha256=4b1de9fd1c59b757cac928b91453df65a5729560448a5b205c195e0164e8ba0a
  Stored in directory: /root/.cache/pip/wheels/5c/1c/15/f037ee255bad89d0652e4238714086f405990a6687d5148f99
Successfully built utils2
     |████████████████████████████████| 6.8MB 9.0MB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [4]:
!python -m pip show scikit-learn 

Name: scikit-learn
Version: 0.23.2
Summary: A set of python modules for machine learning and data mining
Home-page: http://scikit-learn.org
Author: None
Author-email: None
License: new BSD
Location: /usr/local/lib/python3.6/dist-packages
Requires: threadpoolctl, scipy, numpy, joblib
Required-by: yellowbrick, umap-learn, textgenrnn, sklearn, sklearn-pandas, mlxtend, lucid, lightgbm, librosa, imbalanced-learn, fancyimpute


In [5]:
#Tools
import importlib
import utils2
from utils2 import *
import os 
import cv2
import json
from multiprocessing import Process
from multiprocessing import Queue

# Libs
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


#tf
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import datasets, layers, models

#Keras 
import keras as keras
from keras import regularizers
from keras.layers import concatenate
from keras.models import Model
from keras_tqdm import TQDMNotebookCallback
from keras.models import load_model
from keras.layers.merge import add, concatenate, Multiply
from keras import regularizers
from keras.utils import to_categorical


#sklearn
import sklearn.model_selection
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

#SciPy
from scipy.ndimage import gaussian_filter
from scipy.ndimage import map_coordinates

## Import Data 

In [6]:
PATH = ''

In [7]:
def load_data():
    (train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()
    return (train_images, train_labels), (test_images, test_labels)

In [8]:
 (x_train, y_train), (x_test, y_test) = load_data()

170500096/170498071 [==============================] - 4s 0us/step


In [9]:
print(y_train.shape)

(50000, 1)


In [10]:
class Dataset:
    def __init__(self, name, preprocess = False, amplitude = [5], stdDev = [2], rotate_augment = False, angles = [60]):
        self.name = name
        self.isPreprocessed = preprocess
        self.amplitude = amplitude
        self.stdDev = stdDev
        self.angles = angles
        self.IMAGE_SIZE_CROP = 32
        self.IMAGE_SIZE = 100
        
        (x_train, y_train), (x_test, y_test) = load_data()
        
        x_train = x_train/255.
        x_test = x_test/255.
        
        self.X_train = x_train
        
        self.oneHotTransformer = OneHotEncoder(handle_unknown='ignore', sparse = False)
        
        self.Y_train = self.oneHotTransformer.fit_transform(y_train.reshape(-1, 1))
        self.y_test = self.oneHotTransformer.fit_transform(y_test.reshape(-1, 1))
    
        self.splitData()
        
        if rotate_augment == True:
            print("Augmentation of the dataset by rotation")
            Y_list = []
            q = Queue()
            p_ = []
            for angle in self.angles:
                p = Process(target=self.applyRotationToDataset, args=(X_train_original, angle, q,))
                p_.append(p)
                p.start()
                print('*', end='')
                Y = np.asarray(list(Y_train_original))
                Y_list.append(Y)
                print('\n preparing for join')
      
            for i in range(0,len(Y_list)):
                self.X_train = np.asarray(list(self.X_train) + list(q.get()))
                self.Y_train = np.asarray( list(self.Y_train) + list(Y_list[i]))

            for p in p_:
                p.join()
                print('*', end='')
        
    def convertData(self, data):
        images = []
        for dataLine in data:
            if self.isPreprocessed == False:
                images.append(np.reshape(dataLine[1], (self.IMAGE_SIZE, self.IMAGE_SIZE, 1)))
            else:
                images.append(np.reshape(dataLine, (self.IMAGE_SIZE_CROP, self.IMAGE_SIZE_CROP, 1)))
        
        return np.asarray(images)

    def visualizeData(self, indexBegin, indexEnd, dataset = 'X_train'):
        if dataset == 'X_train':
            data = self.X_train
        elif dataset == 'X_valid':
            data = self.X_valid
        elif dataset == 'X_test':
            data = self.X_test
        elif dataset == 'X_submission':
            data = self.X_submission
    
        for index in range(indexBegin, indexEnd):
            plt.imshow(np.reshape(data[index], (self.IMAGE_SIZE_CROP,self.IMAGE_SIZE_CROP)), cmap='Greys')
            plt.show()

    def splitData(self):
        self.X_train, self.X_valid, self.Y_train, self.Y_valid = train_test_split(self.X_train, self.Y_train, test_size=0.20, random_state=42)
  
    def applyRotationToDataset(self, dataset, angle, q):
        rotated_dataset = []
        for img in dataset:
            img = np.reshape(img, (self.IMAGE_SIZE_CROP, self.IMAGE_SIZE_CROP))
            img = self.rotate_image(img, angle)
            rotated_dataset.append(np.reshape(img, (self.IMAGE_SIZE_CROP, self.IMAGE_SIZE_CROP, 1)))
        q.put(np.asarray(rotated_dataset))

    def rotate_image(self, image, angle):
        center = tuple(np.array(image.shape[1::-1]) / 2)
        matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
        result = cv2.warpAffine(image, matrix, image.shape[1::-1], flags=cv2.INTER_LINEAR)
        return result


In [11]:
dataset = Dataset('Data', False, amplitude = None, stdDev = None, rotate_augment = False, angles = None)

In [12]:
print(dataset.Y_train[1])

[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]


## Model 

In [13]:
class TrainableModel:
    def __init__(self, dataset, model, name = "", patience = 15):
        self.name = name
        self.dataset = dataset
        self.model = model
        self.es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=patience)
        self.mc = ModelCheckpoint(PATH + 'best_model.h5', monitor='val_loss', mode='min', save_best_only=True)
        self.optimizer = 'adam'
        self.compileModel()

    def evaluate(self):
        #self.loadBestModel()
        results = self.model.evaluate(self.dataset.X_test, self.dataset.Y_test, batch_size=128)
        self.test_accuracy = results[1]
        print(results)
        with open(PATH + 'History_' + str(self.name) + '_' + str(self.dataset.name) + '_' + str(self.test_accuracy) + '.json', 'w') as f:
            json.dump(self.history.history, f)
    
    def compileModel(self):
        self.model.compile(optimizer= self.optimizer, loss=tf.keras.losses.categorical_crossentropy, metrics=['accuracy'])
    
    def softmaxVectorToOneHot(self, vector):
        index = np.argmax(vector, axis=0)
        oneHot = np.zeros(31)
        oneHot[index] = 1
        return oneHot

    def submission(self):
        #self.loadBestModel()
        Y_predicted = self.model.predict(self.dataset.X_submission)
        labels = []
        for line in Y_predicted:
            labels.append(self.dataset.oneHotTransformer.inverse_transform(self.softmaxVectorToOneHot(line).reshape(1, -1))[0][0])
        df = pd.DataFrame()
        df['Category'] = labels
        df.insert(0, 'Id', range(0, len(df)))
        df.to_csv(PATH + 'Submission-' + str(self.test_accuracy) + '.csv',index=False)

    def fit(self, num_epochs = 300, batchSize = 16):
        X = self.dataset.X_train
        Y = self.dataset.Y_train
        X_valid = self.dataset.X_valid
        Y_valid = self.dataset.Y_valid
        self.history = self.model.fit(X, Y, epochs=num_epochs, batch_size = batchSize, validation_data=(X_valid, Y_valid), callbacks=[self.es, self.mc])

    def loadBestModel(self):
        self.model = load_model(PATH + 'best_model.h5')

    def fitOverAllData(self, num_epochs = 5):
        X = self.dataset.X_ALL
        Y = self.dataset.Y_ALL
        self.history = self.model.fit(X, Y, epochs=num_epochs)

    def graphFittingLoss(self):
        plt.plot(self.history.history['loss'])
        plt.plot(self.history.history['val_loss'])
        plt.title('Model loss over the training steps')
        plt.ylabel('loss')
        plt.xlabel('epoch')
        plt.legend(['train', 'test'], loc='upper left')
        plt.show()

    def graphFittingAccuracy(self):
        plt.plot(self.history.history['accuracy'])
        plt.plot(self.history.history['val_accuracy'])
        plt.title('Model accuracy over the training steps')
        plt.ylabel('accuracy')
        plt.xlabel('epoch')
        plt.legend(['train', 'test'], loc='upper left')

## Dense Net 

In [14]:
RUN = True

In [15]:
#This implementation of the DenseNet architecture is based on the fastai course
#https://github.com/fastai/courses/blob/master/deeplearning2/densenet-keras.ipynb

def convolution(x, numberFilters, filterSize, weigthDecay, p):
    x = layers.Conv2D(numberFilters, (filterSize, filterSize), kernel_initializer='uniform', padding='same', kernel_regularizer = tf.keras.regularizers.l2(weigthDecay))(x)
    return layers.Dropout(p)(x) if p else x

def convolutionBlock(x, numberFilters, bottleneck=False, p=None, weigthDecay=0):
    x = layers.Activation('relu')(layers.BatchNormalization(axis=-1)(x))
    if bottleneck: x = layers.Activation('relu')(layers.BatchNormalization(axis=-1)(convolution(x, numberFilters * 4, 1, weigthDecay, p)))
    return convolution(x, numberFilters, 3, weigthDecay, p)

def denselyConnectedBlock(x, numberOfLayers, growthRate, bottleneck=False, p=None, weigthDecay=0):
    if bottleneck: numberOfLayers //= 2
    for i in range(numberOfLayers):
        b = convolutionBlock(x, growthRate, bottleneck=bottleneck, p=p, weigthDecay=weigthDecay)
        x = concatenate([x,b], -1)
    return x

def TransBlock(x, compression=1.0, p=None, weigthDecay=0):
    numberFilters = int(x.get_shape().as_list()[-1] * compression)
    x = layers.Activation('relu')(layers.BatchNormalization(axis=-1)(x))
    x = convolution(x, numberFilters, 1, weigthDecay, p)
    return layers.AveragePooling2D((2, 2), strides=(2, 2))(x)

def Dense_Net_INIT(input, numberOfLabels, depth=40, blockCount=3, growthRate=12, filterCount=16, bottleneck=False, compression=1.0, p=None, weigthDecay=0):
    numberOfLayers = [int((depth - 4) / blockCount)] * blockCount

    x = convolution(input, filterCount, 3, weigthDecay, 0)
    for i,block in enumerate(numberOfLayers):
        x = denselyConnectedBlock(x, block, growthRate, bottleneck=bottleneck, p=p, weigthDecay=weigthDecay)
        if i != len(numberOfLayers)-1:
            x = TransBlock(x, compression=compression, p=p, weigthDecay=weigthDecay)

    x = layers.Activation('relu')(layers.BatchNormalization(axis=-1)(x))
    x = layers.GlobalAveragePooling2D()(x)
    return layers.Dense(numberOfLabels, activation='softmax', kernel_regularizer=regularizers.l2(weigthDecay))(x)

In [16]:
input = layers.Input(shape=(32, 32, 3))
output = Dense_Net_INIT(input, 10, depth = 100, growthRate=36)
model = models.Model(input, output)

In [ ]:
if RUN:
    denseNet = TrainableModel(dataset, model, name = "DenseNet")
    denseNet.optimizer = tf.keras.optimizers.SGD(0.1, 0.9, nesterov=True)
    denseNet.compileModel()
    denseNet.fit(num_epochs = 100)
    denseNet.evaluate()
    denseNet.submission()

Epoch 1/100
2500/2500 [==============================] - 1672s 669ms/step - loss: 1.8133 - accuracy: 0.3853 - val_loss: 1.4390 - val_accuracy: 0.5139
Epoch 2/100
2500/2500 [==============================] - 1672s 669ms/step - loss: 1.1764 - accuracy: 0.5813 - val_loss: 1.3135 - val_accuracy: 0.6070
Epoch 3/100
2500/2500 [==============================] - 1673s 669ms/step - loss: 0.8934 - accuracy: 0.6867 - val_loss: 0.9365 - val_accuracy: 0.6856
Epoch 4/100
2500/2500 [==============================] - 1679s 671ms/step - loss: 0.7144 - accuracy: 0.7519 - val_loss: 0.7266 - val_accuracy: 0.7506
Epoch 5/100
2500/2500 [==============================] - 1679s 671ms/step - loss: 0.5905 - accuracy: 0.7926 - val_loss: 0.5825 - val_accuracy: 0.7990
Epoch 6/100
2500/2500 [==============================] - 1679s 672ms/step - loss: 0.4967 - accuracy: 0.8267 - val_loss: 0.8958 - val_accuracy: 0.7241
Epoch 7/100
2500/2500 [==============================] - 1679s 671ms/step - loss: 0.4957 - accuracy: